# Fouille données

In [ ]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn mlxtend

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import time
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from folium.plugins import FastMarkerCluster
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from scipy.spatial import ConvexHull

%matplotlib inline
sns.set(style="whitegrid")

# Importation des données

In [ ]:
df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")
print("Nombre de photos au départ :  " + str(len(df)))

# Strip whitespace from column names immediately after loading
df.columns = df.columns.str.strip()
print("Colonnes nettoyées (espaces supprimés)")

# Analyse des données

In [ ]:
# Check available columns
print("Available columns in DataFrame:")
print(df.columns.tolist())
print(f"\nDataFrame shape: {df.shape}")
print("\nFirst few rows:")
display(df.head())

In [ ]:
# Vérification et affichage des données incohérentes dans df original (avant nettoyage)
print("Filtrage des données avec dates incohérentes :")
condition_invalide = (
    (df['date_taken_hour'] < 0) | (df['date_taken_hour'] > 23) |
    (df['date_taken_month'] < 1) | (df['date_taken_month'] > 12) |
    (df['date_taken_day'] < 1) | (df['date_taken_day'] > 31)
)
df_incoherentes = df[condition_invalide]
print(f"Nombre total de données incohérentes : {len(df_incoherentes)}")
print("Affichage du tableau des données incohérentes :")
display(df_incoherentes)

In [ ]:
# Analyse profonde des données incohérentes
print("Analyse des données incohérentes :")
print(f"Colonnes disponibles : {list(df_incoherentes.columns)}")

# Statistiques descriptives des colonnes de date pour les incohérentes
print("\nStatistiques des colonnes de date dans les données incohérentes :")
date_cols = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']
print(df_incoherentes[date_cols].describe())

# Vérifier les types de données
print("\nTypes de données dans df_incoherentes :")
print(df_incoherentes[date_cols].dtypes)

# Examiner quelques exemples spécifiques
print("\nExemples de lignes incohérentes (premières 10) :")
display(df_incoherentes[date_cols].head(10))

# Chercher des patterns : par exemple, heures >23, voir si elles pourraient être des minutes
heures_sup_23 = df_incoherentes[df_incoherentes['date_taken_hour'] > 23]
print(f"\nLignes avec heure >23 : {len(heures_sup_23)}")
if len(heures_sup_23) > 0:
    print("Valeurs d'heure >23 :")
    print(heures_sup_23['date_taken_hour'].unique())
    print("Exemples :")
    display(heures_sup_23[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Mois >12
mois_sup_12 = df_incoherentes[df_incoherentes['date_taken_month'] > 12]
print(f"\nLignes avec mois >12 : {len(mois_sup_12)}")
if len(mois_sup_12) > 0:
    print("Valeurs de mois >12 :")
    print(mois_sup_12['date_taken_month'].unique())
    print("Exemples :")
    display(mois_sup_12[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Jours >31
jours_sup_31 = df_incoherentes[df_incoherentes['date_taken_day'] > 31]
print(f"\nLignes avec jour >31 : {len(jours_sup_31)}")
if len(jours_sup_31) > 0:
    print("Valeurs de jour >31 :")
    print(jours_sup_31['date_taken_day'].unique())
    print("Exemples :")
    display(jours_sup_31[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Vérifier si les valeurs semblent décalées (par exemple, heure dans minute, etc.)
print("\nVérification de décalage possible :")
# Par exemple, si heure >23, voir si minute est valide
if len(heures_sup_23) > 0:
    print("Pour les heures >23, vérifier les minutes :")
    print(heures_sup_23['date_taken_minute'].describe())

# De même pour mois
if len(mois_sup_12) > 0:
    print("Pour les mois >12, vérifier les jours :")
    print(mois_sup_12['date_taken_day'].describe())

# Chercher des NaN ou valeurs manquantes
print(f"\nNombre de NaN par colonne dans les incohérentes :")
print(df_incoherentes[date_cols].isnull().sum())

# Voir si les valeurs sont des floats ou strings étranges
print("\nExemples de valeurs uniques pour diagnostiquer :")
for col in date_cols:
    unique_vals = df_incoherentes[col].dropna().unique()
    print(f"{col} : {unique_vals[:10]}... (total unique: {len(unique_vals)})")  # Premiers 10 pour aperçu

# Nettoyage des données

In [ ]:
df.columns = df.columns.str.strip()

df_clean = df.dropna(axis=1, how='all')

cols_date = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']

df_clean = df_clean.drop_duplicates(subset=['user', 'lat', 'long'] + cols_date)
df_clean = df_clean.dropna(subset=cols_date)

for col in cols_date:
    df_clean[col] = df_clean[col].astype(int)

colonnes_utiles = ['id', 'user', 'lat', 'long', 'tags', 'title'] + cols_date
df_clean = df_clean[colonnes_utiles]

print("\nDonnées nettoyées :")
display(df_clean.head())
print("Nombre de photos restantes : " + str(len(df_clean)))

pourcentage_supprimes = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"Pourcentage de données supprimées : {pourcentage_supprimes:.2f}%")

In [ ]:
print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")

df_clean = df_clean[
    (df_clean['date_taken_year'] >= 2004) &
    (df_clean['date_taken_year'] <= 2026)
]

print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")
print("Après filtre temporel : " + str(len(df_clean)) + " photos.")

In [ ]:
# Vérification des valeurs invalides dans les colonnes de date
print("Vérification des heures invalides (pas entre 0 et 23) :")
heures_invalides = df_clean[(df_clean['date_taken_hour'] < 0) | (df_clean['date_taken_hour'] > 23)]
print(f"Nombre d'heures invalides : {len(heures_invalides)}")
if len(heures_invalides) > 0:
    display(heures_invalides[['id', 'date_taken_hour']].head(10))  # Afficher les 10 premières pour lisibilité

print("\nVérification des mois invalides (pas entre 1 et 12) :")
mois_invalides = df_clean[(df_clean['date_taken_month'] < 1) | (df_clean['date_taken_month'] > 12)]
print(f"Nombre de mois invalides : {len(mois_invalides)}")
if len(mois_invalides) > 0:
    display(mois_invalides[['id', 'date_taken_month']].head(10))

print("\nVérification des jours invalides (pas entre 1 et 31) :")
jours_invalides = df_clean[(df_clean['date_taken_day'] < 1) | (df_clean['date_taken_day'] > 31)]
print(f"Nombre de jours invalides : {len(jours_invalides)}")
if len(jours_invalides) > 0:
    display(jours_invalides[['id', 'date_taken_day']].head(10))

In [ ]:
# Tentative de correction automatique du décalage de colonnes
def corriger_decalage(row):
    # Copie des valeurs actuelles
    y, m, d, h, mn = row['date_taken_year'], row['date_taken_month'], row['date_taken_day'], row['date_taken_hour'], row['date_taken_minute']

    # Si minute est une année (entre 2000 et 2030), décalage vers la droite détecté
    if pd.notna(mn) and 2000 <= mn <= 2030:
        # Décalage : year <- minute, minute <- hour, hour <- day, day <- month, month <- year
        new_y = int(mn)
        new_mn = h % 60  # minute de 0-59
        new_h = d % 24   # heure de 0-23
        new_d = m % 31   # jour approximatif 1-31
        new_m = y % 12   # mois 1-12
        if new_m == 0: new_m = 12
        if new_d == 0: new_d = 1
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si month >12 et semble être une année, décalage différent
    elif m > 12 and m < 2030:
        # Suppose month est year, alors décalage inverse
        new_y = int(m)
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_d = d % 31
        if new_d == 0: new_d = 1
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si day >31 et semble année
    elif d > 31 and d < 2030:
        new_y = int(d)
        new_d = m % 31
        if new_d == 0: new_d = 1
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Sinon, essayer de corriger seulement les invalides
    else:
        new_y = y
        new_m = m if 1 <= m <= 12 else (m % 12) if m % 12 != 0 else 12
        new_d = d if 1 <= d <= 31 else (d % 31) if d % 31 != 0 else 1
        new_h = h if 0 <= h <= 23 else (h % 24)
        new_mn = mn if pd.notna(mn) and 0 <= mn <= 59 else (mn % 60) if pd.notna(mn) else 0
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

# Appliquer la correction
df_incoherentes_corrected = df_incoherentes.copy()
df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']] = df_incoherentes.apply(corriger_decalage, axis=1)

print("Après correction automatique :")
print("Exemples corrigés :")
display(df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

# Vérifier combien sont maintenant valides
condition_valide_apres = (
    (df_incoherentes_corrected['date_taken_hour'] >= 0) & (df_incoherentes_corrected['date_taken_hour'] <= 23) &
    (df_incoherentes_corrected['date_taken_month'] >= 1) & (df_incoherentes_corrected['date_taken_month'] <= 12) &
    (df_incoherentes_corrected['date_taken_day'] >= 1) & (df_incoherentes_corrected['date_taken_day'] <= 31)
)
valides_apres = df_incoherentes_corrected[condition_valide_apres]
print(f"\nNombre de lignes maintenant valides après correction : {len(valides_apres)} / {len(df_incoherentes_corrected)}")

# Afficher les lignes corrigées valides
if len(valides_apres) > 0:
    print("Exemples de lignes corrigées et maintenant valides :")
    display(valides_apres[['id', 'date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

In [ ]:
# Vérifier que les coordonnées géographiques sont cohérentes.
photos_avant_filtre = len(df_clean)

df_clean = df_clean[
    (df_clean['lat'] >= -90) & (df_clean['lat'] <= 90) &
    (df_clean['long'] >= -180) & (df_clean['long'] <= 180)
]
photos_supprimees = photos_avant_filtre - len(df_clean)
print(f"Photos supprimées par le filtre géographique : {photos_supprimees}")
print(f"Photos avec coordonnées valides : {len(df_clean)}")
print(f"Min/Max Latitude  : {df_clean['lat'].min()} / {df_clean['lat'].max()}")
print(f"Min/Max Longitude : {df_clean['long'].min()} / {df_clean['long'].max()}")

In [ ]:
# Tentative de correction automatique du décalage de colonnes
def corriger_decalage(row):
    # Copie des valeurs actuelles
    y, m, d, h, mn = row['date_taken_year'], row['date_taken_month'], row['date_taken_day'], row['date_taken_hour'], row['date_taken_minute']

    # Si minute est une année (entre 2000 et 2030), décalage vers la droite détecté
    if pd.notna(mn) and 2000 <= mn <= 2030:
        # Décalage : year <- minute, minute <- hour, hour <- day, day <- month, month <- year
        new_y = int(mn)
        new_mn = h % 60  # minute de 0-59
        new_h = d % 24   # heure de 0-23
        new_d = m % 31   # jour approximatif 1-31
        new_m = y % 12   # mois 1-12
        if new_m == 0: new_m = 12
        if new_d == 0: new_d = 1
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si month >12 et semble être une année, décalage différent
    elif m > 12 and m < 2030:
        # Suppose month est year, alors décalage inverse
        new_y = int(m)
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_d = d % 31
        if new_d == 0: new_d = 1
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si day >31 et semble année
    elif d > 31 and d < 2030:
        new_y = int(d)
        new_d = m % 31
        if new_d == 0: new_d = 1
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Sinon, essayer de corriger seulement les invalides
    else:
        new_y = y
        new_m = m if 1 <= m <= 12 else (m % 12) if m % 12 != 0 else 12
        new_d = d if 1 <= d <= 31 else (d % 31) if d % 31 != 0 else 1
        new_h = h if 0 <= h <= 23 else (h % 24)
        new_mn = mn if pd.notna(mn) and 0 <= mn <= 59 else (mn % 60) if pd.notna(mn) else 0
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

# Appliquer la correction
df_incoherentes_corrected = df_incoherentes.copy()
df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']] = df_incoherentes.apply(corriger_decalage, axis=1)

print("Après correction automatique :")
print("Exemples corrigés :")
display(df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

# Vérifier combien sont maintenant valides
condition_valide_apres = (
    (df_incoherentes_corrected['date_taken_hour'] >= 0) & (df_incoherentes_corrected['date_taken_hour'] <= 23) &
    (df_incoherentes_corrected['date_taken_month'] >= 1) & (df_incoherentes_corrected['date_taken_month'] <= 12) &
    (df_incoherentes_corrected['date_taken_day'] >= 1) & (df_incoherentes_corrected['date_taken_day'] <= 31)
)
valides_apres = df_incoherentes_corrected[condition_valide_apres]
print(f"\nNombre de lignes maintenant valides après correction : {len(valides_apres)} / {len(df_incoherentes_corrected)}")

# Afficher les lignes corrigées valides
if len(valides_apres) > 0:
    print("Exemples de lignes corrigées et maintenant valides :")
    display(valides_apres[['id', 'date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

In [ ]:
# Nettoyage supplémentaire : suppression de doublons spatiaux par utilisateur (rayon 2m)
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Distance en mètres approximée
    R = 6371000  # Rayon de la Terre en mètres
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def supprimer_doublons_spatiaux_par_user(df, rayon_m=2):
    df_result = []
    for user, group in df.groupby('user'):
        group = group.sort_values('id')
        garde = []
        for idx, row in group.iterrows():
            lat, lon = row['lat'], row['long']
            trop_proche = False
            for g_lat, g_lon in garde:
                if haversine_distance(lat, lon, g_lat, g_lon) < rayon_m:
                    trop_proche = True
                    break
            if not trop_proche:
                garde.append((lat, lon))
                df_result.append(row)
    return pd.DataFrame(df_result)

photos_avant_spatial = len(df_clean)
print(f"Photos avant nettoyage spatial : {photos_avant_spatial}")

df_clean = supprimer_doublons_spatiaux_par_user(df_clean, rayon_m=2)

photos_apres_spatial = len(df_clean)
photos_supprimees_spatial = photos_avant_spatial - photos_apres_spatial
print(f"Photos supprimées par nettoyage spatial : {photos_supprimees_spatial}")
print(f"Photos restantes après nettoyage spatial : {photos_apres_spatial}")
print(f"Pourcentage supprimé par spatial : {photos_supprimees_spatial / photos_avant_spatial * 100:.2f}%")

# Affichage de la carte

In [ ]:
# Création de la carte centrée sur une latitude et une longitude moyennes
m = folium.Map(location=[45.7640, 4.8357], zoom_start=13)
locations = list(zip(df['lat'], df['long']))
FastMarkerCluster(data=locations).add_to(m)
# Sauvegarde la carte dans un fichier HTML
m.save('ma_carte_lyon_all_photos.html')

m

# Clustering

In [ ]:
df_clustering = df_clean[['lat', 'long']]

In [ ]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering)

# show
scaled_data_df = pd.DataFrame(data=scaled_data, columns=['lat', 'long'])

print("Aperçu des données standardisées (prêtes pour l'algo) :")
display(scaled_data_df.head())

print(f"Moyennes : \n{scaled_data_df.mean()}")

## Estimation du nombre de clusters pour K-means via la méthode du coude 

In [ ]:
inertia = []
k_range = range(1, 50)

for k in k_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(scaled_data_df)
    inertia.append(kmeans.inertia_)

# Création du graphique
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_range)
plt.grid(True)
plt.show()

Au vu des résultats, nous pouvons partir sur une valeur entre 15 et 20. Cela permet d'avoir suffisant de zones d'intérêt pour une grande ville comme Lyon, tout en évitant de diviser les zones d'intéret en plusieurs cluster. Nous choisisons donc une valeur de 17 clusters.

## Utilisation de l'algorithme K-means

In [ ]:
kmeans = KMeans(n_clusters=100, init='k-means++', random_state=42)

kmeans.fit(scaled_data_df)

df_clean['cluster'] = kmeans.labels_

df_clean.head()

## Affichage des clusters sur la carte

In [ ]:


def generer_carte_clusters(df, nom_fichier="ma_carte.html"):
    """
    Génère une carte Folium avec des zones colorées selon la densité (Convex Hull).
    """
    
    filtered_data = df[df['cluster'] != -1]
    
    if filtered_data.empty:
        print("⚠️ Attention : Aucun cluster valide trouvé (tout est bruit ou vide).")
        return None

    counts = filtered_data['cluster'].value_counts()
    
    if len(counts) == 0:
        print("Pas de données à afficher.")
        return None
        
    min_val = counts.min()
    max_val = counts.max()
    
    print(f"--- Génération de {nom_fichier} ---")
    print(f"Cluster min : {min_val} photos | Cluster max : {max_val} photos")

    colors = ["white", "blue", "red", "darkred"]
    cmap = mcolors.LinearSegmentedColormap.from_list("mon_degrade", colors)
    
    # Normalisation
    if min_val == max_val:
        norm = mcolors.Normalize(vmin=min_val, vmax=max_val)
    else:
        norm = mcolors.LogNorm(vmin=min_val, vmax=max_val)

    def get_color(n):
        return mcolors.to_hex(cmap(norm(n)))

    center_lat = df['lat'].mean()
    center_long = df['long'].mean()
    m = folium.Map(location=[center_lat, center_long], zoom_start=13)

    clusters_ids = sorted(df['cluster'].unique())

    for cluster_id in clusters_ids:
        if cluster_id == -1: 
            continue

        points = df[df['cluster'] == cluster_id][['lat', 'long']].values
        n_photos = len(points)
        
        color = get_color(n_photos)
        
        opacity = 0.3 + (norm(n_photos) * 0.5) if max_val > min_val else 0.5

        if len(points) >= 3:
            try:
                hull = ConvexHull(points)
                points_contour = points[hull.vertices]
                
                folium.Polygon(
                    locations=points_contour,
                    color=color,
                    weight=2,
                    fill=True,
                    fill_color=color,
                    fill_opacity=opacity,
                    popup=f"Zone {cluster_id}: {n_photos} photos"
                ).add_to(m)
                
                center = np.mean(points, axis=0)
                style_texte = "color: black; text-shadow: 1px 1px 0px white; font-weight: bold; font-size: 10pt;"
                folium.Marker(
                    center,
                    icon=folium.DivIcon(html=f'<div style="{style_texte}">{n_photos}</div>')
                ).add_to(m)
                
            except Exception:
                pass
                
        else:
            for pt in points:
                folium.CircleMarker(
                    pt, 
                    radius=5, 
                    color=color, 
                    fill=True, 
                    fill_color=color,
                    popup=f"Cluster {cluster_id}"
                ).add_to(m)

    m.save(nom_fichier)
    print(f"Carte sauvegardée avec succès : {nom_fichier}\n")
    return m

In [ ]:
generer_carte_clusters(df_clean, "carte_k_means.html")

On voit que certaines zones sont bien plus denses que d'autres. En tant que connaisseurs de la ville de Lyon, nous reconnaissons bien évidemment que que le centre-ville et le vieux-Lyon sont les endroits les plus photographiés, ce qui parait cohérent. 

Cependant, K-means nous permet seulement de voir de grandes zones, ce qui n'est clairement pas ce que nous cherchons à faire pour déterminer des petits clusters d'évènements. Nous devons trouver d'autres algorithmes.

## Utilisation de l'algorithme Hierarchical clustering

In [ ]:
subset_df = scaled_data_df.sample(n=3000, random_state=42)

print(f"Recherche des paramètres optimaux sur {len(subset_df)} points...")

linkage_methods = ['ward', 'average']
range_n_clusters = range(10, 1000, 10)

scores = {'ward': [], 'average': []}

for method in linkage_methods:
    print(f"Test de la méthode : {method}...")
    for k in range_n_clusters:
        model = AgglomerativeClustering(n_clusters=k, linkage=method)        
        labels = model.fit_predict(subset_df)
        
        # On calcule le score pour déterminer la qualité de la séparation
        score = silhouette_score(subset_df, labels)
        scores[method].append(score)

plt.figure(figsize=(12, 6))

# Courbe Ward
plt.plot(range_n_clusters, scores['ward'], label='Ward (Compact)', marker='o', color='blue')
# Courbe Average
plt.plot(range_n_clusters, scores['average'], label='Average (Naturel)', marker='x', color='green', linestyle='--')

plt.title("Comparaison ward vs Average")
plt.xlabel("Nombre de Clusters (k)")
plt.ylabel("Silhouette Score")
plt.legend()
plt.grid(True)
plt.show()

# meilleurs résulats
best_ward_score = max(scores['ward'])
best_ward_k = range_n_clusters[scores['ward'].index(best_ward_score)]

print(f"Meilleur config Ward : k={best_ward_k} (Score: {best_ward_score:.3f})")

In [ ]:


LINKAGE_GAGNANT = 'ward' 
N_CLUSTERS = 250

print(f"Lancement du Clustering Hiérarchique ({LINKAGE_GAGNANT}) pour {N_CLUSTERS} zones...")

print("Calcul du graphe de connectivité...")
connectivity = kneighbors_graph(scaled_data_df, n_neighbors=10, include_self=False)

model = AgglomerativeClustering(
    n_clusters=N_CLUSTERS, 
    connectivity=connectivity, 
    linkage=LINKAGE_GAGNANT
)

print("Entraînement du modèle (patience)...")
start = time.time()
# fit_predict calcule les groupes directement
labels = model.fit_predict(scaled_data_df) 
end = time.time()
print(f"Terminé en {end - start:.2f} secondes !")

df_clean['cluster'] = labels

nom_fichier = f"carte_hierarchique_{LINKAGE_GAGNANT}.html"
generer_carte_clusters(df_clean, nom_fichier)

## Utilisation de l'algorithme DBSCAN 

In [ ]:
min_samples = 30

# On regarde la distance vers le k-ième voisin
neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(scaled_data_df)
distances, indices = neighbors_fit.kneighbors(scaled_data_df)

distances = np.sort(distances[:, min_samples-1], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.title("Graphe des k-distances (pour trouver eps)")
plt.xlabel("Points triés")
plt.ylabel("Epsilon (Distance)")
plt.grid(True)
plt.show()

In [ ]:
min_samples = range(30, 80, 5)
eps = np.arange(0.002, 0.06, 0.002)
output = []

for ms in min_samples:
    for ep in eps:
        labels = DBSCAN(min_samples=ms, eps = ep).fit(scaled_data_df).labels_
        score = silhouette_score(scaled_data_df, labels, sample_size=5000, random_state=42)
        output.append((ms, ep, score))


min_samples, eps, score = sorted(output, key=lambda x:x[-1])[-1]
print(f"Best silhouette_score: {score}")
print(f"min_samples: {min_samples}")
print(f"eps: {eps}")

Au vu du résultat lorsque nous utilisons ces valeurs, il semble que le epsilon est trop grand. En effet, nous avons encore des clusters trop grands. Cependant le min_samples semble cohérent. Pour pouvoir différencier clairement les zones d'affluence, il faudrait un epsilon bien plus petit. 

In [ ]:
EPSILON_CHOISI = 0.058
MIN_SAMPLES = 45

print(f"Lancement de DBSCAN (eps={EPSILON_CHOISI}, min={MIN_SAMPLES})...")

dbscan = DBSCAN(eps=EPSILON_CHOISI, min_samples=MIN_SAMPLES)
clusters = dbscan.fit_predict(scaled_data_df)

df_clean['cluster'] = clusters

n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
len(clusters)
n_noise = list(clusters).count(-1)
percent_noise = (n_noise / len(df_clean)) * 100

print(f"--> Résultat : {n_clusters} clusters trouvés.")
print(f"--> Bruit : {n_noise} photos ignorées ({percent_noise:.1f}%)")

nom_fichier = f"carte_dbscan.html"
generer_carte_clusters(df_clean, nom_fichier)

## Utilisation de l'algorithme Apriori sur les tags 

In [ ]:
# Récupérer les clusters uniques (exclure le bruit -1 si présent)
clusters_uniques = sorted([c for c in df_clean['cluster'].unique() if c != -1])

print(f"\nAnalyse Apriori pour {len(clusters_uniques)} clusters\n")
print("="*80)

resultats_apriori = {}

for cluster_id in clusters_uniques:
    print(f"Cluster {cluster_id}...", end=" ", flush=True)
    
    # Filtrer les données du cluster
    cluster_data = df_clean[df_clean['cluster'] == cluster_id]
    
    if len(cluster_data) == 0:
        print("vide")
        continue
    
    # Créer liste de transactions (tags par photo)
    transactions = []
    for tags_str in cluster_data['tags']:
        if pd.notna(tags_str) and tags_str != '':
            tags_list = [tag.strip() for tag in str(tags_str).split(',') if tag.strip()]
            if tags_list:
                transactions.append(tags_list)
    
    if not transactions:
        print("pas de tags")
        continue
    
    # Encoder et appliquer Apriori
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Support plus haut pour éviter la surcharge
    frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)
    
    # Récupérer les tags fréquents
    frequent_words = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) == 1)]
    frequent_words = frequent_words.sort_values('support', ascending=False).head(5)
    
    resultats_apriori[cluster_id] = frequent_words
    
    # Afficher résultats
    print(f"({len(cluster_data)} photos)")
    if len(frequent_words) > 0:
        for _, row in frequent_words.iterrows():
            tag = list(row['itemsets'])[0]
            print(f"  - {tag}: {row['support']:.1%}")

print("="*80)
print(f"\nTerminé: {len(resultats_apriori)} clusters analysés")